### build-mysql-database
CONSTROI BASE DE DADOS NO MYSQL POR MEIO DE DOCUMENTOS DA COLEÇÃO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS INFORMAÇÕES DE CADA DOCUMENTO A SEREM INSERIDAS NA BASE SÃO:
* ID
* CORPO DE TEXTO
* DATAS (ANO E MÊS)
* URL
* IDIOMA PRINCIPAL

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
#import psycopg2
import getpass

from sshtunnel import SSHTunnelForwarder
import time

import codecs
from langdetect import detect
from IPython.display import clear_output
import csv

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
#path = "C:/Users/marcelo.ribeiro/Documents/temp/" #TESTE!
#outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"
#path = "/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/"

files = [f for f in os.listdir(path)]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)

In [3]:
files[0:4]

['AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'AAS_mre_ag_1973.11.20_doc_I-10.txt',
 'AAS_mre_ag_1973.11.20_doc_I-100.txt',
 'AAS_mre_ag_1973.11.20_doc_I-101.txt']

In [4]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria listas de url e dossies correspondentes
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
url_list = []
dossie_list = []
url_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/URLS-AAS_2.csv"
with open(url_inputs, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if ' mre ' not in row[1]: continue
        dossie = re.sub(' ','_',row[1])
        dossie = re.sub('AAS_mre_(.*)',r'\1',dossie)
        dossie_list.append(dossie)
        url_list.append(row[2])

In [5]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria e alimenta a base mysql
Classifica o corpus de acordo com: tipo textual (para separar textos sujos, como letra cursiva, foto, fonte alternativa) e idioma

Textos ilegíveis são retirados da base mysql, mas listados na pasta Outputs para que busquemos soluções futuramente.

pt = 1
en = 2
es = 3
fr = 4
de = 5
não identificado = 0
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
temp = 0

doc_class = []
lang_class = 0
count_doc = 0
not_readable = []
percentil = int(len(fullpath_list)/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_private_key_password = input('Entre com a senha da chave privada'),
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           #passwd=input('Entre com a senha do usuário no banco de dados'),
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE marcelo_draft")
    cur.execute('''CREATE TABLE IF NOT EXISTS marcelo_draft 
               (id VARCHAR(50), body LONGTEXT, year VARCHAR(4), month VARCHAR(2), url VARCHAR(200), main_language VARCHAR(10), temp VARCHAR(2)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    cur.execute("DELETE FROM marcelo_draft WHERE temp = 0")
    
    for txt in fullpath_list[0:200]:
        
        ### mede percentual de conclusão da tarefa ###
        count_doc += 1
        if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
        if count_doc % (percentil-1) == 0: clear_output()
        
        ### captura info de ano, mes e id de cada documento ###
        txt_year = re.sub('.*(19\d\d).*', r'\1', txt)
        txt_month = re.sub('.*19\d\d\.(\d\d).*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)

        ### captura info de url ###
        txt_id_dossie = re.sub('.*AAS_mre_(.*)_doc.*', r'\1', txt)
        for dossie_with_url in dossie_list:
            if txt_id_dossie == dossie_with_url:
                list_number = dossie_list.index(dossie_with_url)
                url = url_list[list_number]
                
        with open(txt, 'r', encoding='utf-8') as f: #obs: deveria usar encoding utf-8. Ignorando erros
            txt_body = f.read()

            ### identifica idioma por documento ###
            texto_pt = texto_en = texto_es = texto_fr = txt_body
            texto_split = re.split('\.|\?|\:', txt_body)
            #print(texto_split)
            count = pt_count = en_count = es_count = fr_count = de_count = lang_count = total_count = 0
            for phrase in texto_split:
                total_count += 1
                if not re.search('[^\W\d_]', phrase): continue
                try: 
                    language = detect(phrase)
                except: 
                    print(txt)
                    print('PHRASE', phrase, 'LANGUAGE', language)
                total_count += 1
                if language == 'pt':
                    pt_count += 1
                if language == 'en':
                    en_count += 1
                if language == 'es':
                    es_count += 1
                if language == 'fr':
                    fr_count += 1
                if language == 'de':
                    de_count += 1
            lang_count = pt_count + en_count + es_count + fr_count + de_count        
            if lang_count/total_count < 0.08: 
                #print('not readable', txt)
                not_readable.append(txt)
                continue
            # obs: provavelmente sobrarão documentos com lang_class = 0. Manter sem identificação de idioma?
            if de_count/total_count > 0.1: 
                lang_class = 5
            if fr_count/total_count > 0.1: 
                lang_class = 4
            if es_count/total_count > 0.1: 
                lang_class = 3
            if en_count/total_count > 0.1: 
                lang_class = 2
            if pt_count/total_count > 0.1: 
                lang_class = 1
            
            ### insere dados na base mysql ###
            query = "INSERT INTO marcelo_draft VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cur.execute(query, (txt_id, txt_body, txt_year, txt_month, url, lang_class, temp))

    cur.execute("SELECT * FROM marcelo_draft");
    rows = cur.fetchall()
    
    print(rows)

1 % done
(('ag_1973.11.20_doc_I-1', ' \n\njr, /h \' "& m$ hb\'h\'qo\n& la\'?\n\n, é %%3 - _\n(aonsulado do brasil _/ , ova/caio\n4 & ,\n* brazilian consulate wo \'\naas hawthorn st. 16 % wja mm\nmu. .. " "\nmm .\n\nnew bedford, massachuseti\'s. u. s, a, 02740\n\n \n\nmarcio muller bueno\n" , /\n11:11:3sz mao jz". www não"3337552 %%, " aca-m\n)\nwha %" wom- aba/và %%\n\nxb" 7/35 andado.\n\n_, íáaw mí. m luz/á && %% %%, aj alma, ;\n(\n\n"me/&& % \'wl-(aajj-mi- # "a" aaa/fo &\n"não. aiwa %%%&/a %%%&/www a* m" á?!" á\n1/1"me %"aí\'ej\'wfo m/k%/ám%â tai/á\nw. : úwá 2-- /i < %%\n"melwa/ volíco ay? mul/1 "( o , (-\n\n\' % ó<\n/l ano)/4% wklt-w wa: (lw/ám/[a w\n&& mwráww/láw wa",\n\n/-\n\númháo , (; vzííóo quo/\'. ,\n"ac-"44. \' pá , (".\nv £ f (1. é : "l 01; mw w wm %- &\n/\n\nwl] [iv-156%; , (, \'\n"ab/q "já-m m€) kífa-nmc aaa-x mw, ala\nwww,\n%"wáwí, 9 m: ( c/mn\n\n%sjjéãm\n\n"ém-k w [\', /á" "**-"í; pé ifá?!" w %\'0\nm amo. .. /lilia"? jmá%. %i/w %\n&& po "vpmtãvak_ jun moo-"n & %% "4/4, "

In [29]:
rows = ()
print(type(rows))

<class 'tuple'>


In [ ]:
for txt in fullpath_list:
    txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
    txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
    with open(os.path.join(path,txt), 'r', encoding='utf-8') as f:
        txt_body = f.read()
        #curs.execute('''INSERT INTO marcelo_draft VALUES (txt_id, txt_body, txt_date, txt_date)''')

In [18]:
#print(not_readable)
file = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/not_readable.txt"
for i in not_readable:
    if i == not_readable[0]: 
        with open(file, 'w', encoding='utf-8') as f: 
            txt = f.write(i)
    else: 
        with open(file, 'a', encoding='utf-8') as f: 
            txt = f.write('\r\n')
            txt = f.write(i)
        
        



In [7]:
print(not_readable[0])

C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-10.txt


### RASCUNHOS

In [28]:
count = 0
for txt in fullpath_list[0:1]:
        count += 1
        txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
        #obs: deveria usar encoding utf-8
        with open(txt, 'rb') as f:
            print(f)
            txt_body = f.read()
            
            txt_body = txt_body.decode('latin-1').encode('utf8')
            print(txt_body)

<_io.BufferedReader name='C:/Users/marcelo.ribeiro/Documents/temp/AAS_mre_ag_1973.11.20_doc_I-1.txt'>
b' \r\n\r\njr, /h \' "& m$ hb\'h\'qo\r\n& la\'?\r\n\r\n, \xc3\xa9 %%3 - _\r\n(aonsulado do brasil _/ , ova/caio\r\n4 & ,\r\n* brazilian consulate wo \'\r\naas hawthorn st. 16 % wja mm\r\nmu. .. " "\r\nmm .\r\n\r\nnew bedford, massachuseti\'s. u. s, a, 02740\r\n\r\n \r\n\r\nmarcio muller bueno\r\n" , /\r\n11:11:3sz mao jz". www n\xc3\xa3o"3337552 %%, " aca-m\r\n)\r\nwha %" wom- aba/v\xc3\xa0 %%\r\n\r\nxb" 7/35 andado.\r\n\r\n_, \xc3\xad\xc3\xa1aw m\xc3\xad. m luz/\xc3\xa1 && %% %%, aj alma, ;\r\n(\r\n\r\n"me/&& % \'wl-(aajj-mi- # "a" aaa/fo &\r\n"n\xc3\xa3o. aiwa %%%&/a %%%&/www a* m" \xc3\xa1?!" \xc3\xa1\r\n1/1"me %"a\xc3\xad\'ej\'wfo m/k%/\xc3\xa1m%\xc3\xa2 tai/\xc3\xa1\r\nw. : \xc3\xbaw\xc3\xa1 2-- /i < %%\r\n"melwa/ vol\xc3\xadco ay? mul/1 "( o , (-\r\n\r\n\' % \xc3\xb3<\r\n/l ano)/4% wklt-w wa: (lw/\xc3\xa1m/[a w\r\n&& mwr\xc3\xa1ww/l\xc3\xa1w wa",\r\n\r\n/-\r\n\r\n\r\n"ac-"44. \' 

In [14]:

codecs.decode('''jr, /h ' "& m$ hb'h'qo & la'?  , é %%3 - _ (aonsulado do brasil _/ , ova/caio 4 & , * brazilian consulate wo ' aas hawthorn st. 16 % wja mm mu. .. " " mm .  new bedford, massachuseti's. u. s, a, 02740     marcio muller bueno " , / 11:11:3sz mao jz". www não"3337552 %%, " aca-m ) wha %" wom- aba/và %%  xb" 7/35 andado.  _, íáaw mí. m luz/á && %% %%, aj alma, ; (  "me/&& % 'wl-(aajj-mi- # "a" aaa/fo & "não. aiwa %%%&/a %%%&/www a* m" á?!" á 1/1"me %"aí'ej'wfo m/k%/ám%â tai/á w. : úwá 2-- /i < %% "melwa/ volíco ay? mul/1 "( o , (-  ' % ó< /l ano)/4% wklt-w wa: (lw/ám/[a w && mwráww/láw wa",  /-   "ac-"44. ' pá , (". v £ f (1. é : "l 01; mw w wm %- & /  wl] [iv-156%; , (, ' "ab/q "já-m m€) kífa-nmc aaa-x mw, ala www, %"wáwí, 9 m: ( c/mn  %sjjéãm  "ém-k w [', /á" "**-"í; pé ifá?!" w %'0 m amo. .. /lilia"? jmá%. %i/w % && po "vpmtãvak_ jun moo-"n & %% "4/4, "  p (aõ piatã h , azo"wat ao.     the secretary of state washington  november 20, 1973  dear mr. bueno:  thank you for your letter of congratulations. i remember our work with the brazilian student-leaders, and i deeply appreciate hearing from you.  best regards,  por"?  henry a. Kissinger  mr. marcio muller bueno, consul of brazil, brazilian consulate, 229 hawthorn street, new bedford, massachusetts.  ;" m ''')
#'\xc4pple'
#apple.decode('iso-8859-1').encode('utf8')

TypeError: decoding with 'utf-8' codec failed (TypeError: a bytes-like object is required, not 'str')